In [1]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using PyCall
using JLD2
using LinearAlgebra
using Rotations
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMol
  ✓ Distributions → DistributionsTestExt
  ✓ KernelDensity
  ✓ Makie
  ✓ CairoMakie
  ✓ MorphoMol
  5 dependencies successfully precompiled in 57 seconds. 239 already precompiled.
[ Info: Precompiling MorphoMol [85e20460-a9b2-48f6-9df6-e154e9748d83]
Precompiling GLMakie
  ✓ MeshIO
  ✓ Distributions → DistributionsChainRulesCoreExt
  ✓ KernelDensity
  ✓ Makie
  ✓ GLMakie
  5 dependencies successfully precompiled in 62 seconds. 236 already precompiled.
[ Info: Precompiling GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a]
┌ Warning: Module Distributions with build ID fafbfcfd-1e9c-0a61-0000-14cf484f9e73 is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969acd0382f] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee3

In [2]:
pyimport("sys").executable

"/Users/ivanspirandelli/Doktor/Code/Notebooks/oineus_venv/bin/python3"

In [3]:
py"""
import oineus as oin
import numpy as np
import torch
import diode

def get_coboundary_diagram(points, n_atoms_per_mol):
    points = np.asarray(points)
    simplices = diode.fill_alpha_shapes(points)
    fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])

    def is_multi(sigma):
        has_a = has_b = False
        for v in sigma.vertices:
            if v < n_atoms_per_mol:
                has_a = True
            else:
                has_b = True
        return has_a and has_b
    fil = fil.subfiltration(is_multi)
    # second argument: True for cohomology, False for homology (incorrect for subfiltration)
    dcmp = oin.Decomposition(fil, True)
    params = oin.ReductionParams()
    params.clearing_opt = False
    dcmp.reduce(params)
    dgm = dcmp.diagram(fil, include_inf_points=False)
    return dgm
"""

function get_coboundary_diagram(points::Vector{Vector{Float64}}, n_atoms_per_mol::Int)
    py"get_coboundary_diagram"(points, n_atoms_per_mol) 
end

get_coboundary_diagram (generic function with 1 method)

In [4]:
perturb_all(x, Σ) = x .+ (randn(length(x)) .* Σ)

function perturb_single_randomly_chosen(x, σ_r, σ_t)
    x_cand = deepcopy(x)
    i  = rand(0:(length(x)÷6)-1)
    x_cand[(i*6)+1:(i*6)+6] = x_cand[(i*6)+1:(i*6)+6] .+ randn(6) .* [σ_r, σ_r, σ_r, σ_t, σ_t, σ_t]
    x_cand
end

function solvation_free_energy_and_measures_in_bounds(x::Vector{Float64}, template_mol::Matrix{Float64}, radii::Vector{Float64}, rs::Float64, prefactors::AbstractVector, overlap_jump::Float64, overlap_slope::Float64, persistence_weight::Float64, bounds::Float64, delaunay_eps::Float64)
    if any(0.0 >= e || e >= bounds for e in x[4:6:end]) || any(0.0 >= e || e >= bounds for e in x[5:6:end]) || any(0.0 >= e || e >= bounds for e in x[6:6:end])
        return Inf, [Inf, Inf, Inf, Inf, Inf]
    end
    n_mol = length(x) ÷ 6
    n_atoms_per_mol = size(template_mol)[2]
    flat_realization = MorphoMol.Utilities.get_flat_realization(x, template_mol)
    cdgms = get_coboundary_diagram(Vector{Vector{Float64}}(eachcol(reshape(flat_realization, (3, Int(length(flat_realization) / 3))))), n_atoms_per_mol)
    cdgms = [cdgms[1], cdgms[2], cdgms[3], cdgms[4]]
    cp2 = sum(cdgms[2][:,2] - cdgms[2][:,1])
    measures = MorphoMol.Energies.get_geometric_measures_and_overlap_value(flat_realization, n_atoms_per_mol, radii, rs, overlap_jump, overlap_slope, delaunay_eps)
    measures = [measures; [cp2]] 
    # TODO: Slightly irritating that we have to set the prefactor for persistence here, while the prefactor for overlap penalty is overlap_slope and passed to measure calc
    sum(measures .* [prefactors; [1.0, persistence_weight]]), Dict("Vs" => measures[1], "As" => measures[2], "Cs" => measures[3], "Xs" => measures[4], "OLs" => measures[5], "CDGMs"  => cdgms, "CP2s" => cp2, "CP3s" => cp3)
end

solvation_free_energy_and_measures_in_bounds (generic function with 1 method)

In [5]:
simulation_time_minutes = 60.0
template_file = "../../Data/jld2/2_6r7m_init/6r7m_protor_20.jld2"

template_mol = MorphoMol.Utilities.TMV_TEMPLATES["6r7m"][1]
template_radii = MorphoMol.Utilities.TMV_TEMPLATES["6r7m"][2]
x_init = MorphoMol.Utilities.get_initial_state(2, 100.0)

T = 2.0
β = 1.0 / T

rs = 1.4
η = 0.3665
pf = MorphoMol.Energies.get_prefactors(rs, η)
overlap_slope = 0.85
persistence_weight = -0.1
bnds = 150.0
delaunay_eps = 1000.0

σ_r = 0.15
σ_t = 1.25

n_atoms_per_mol = length(template_mol) ÷ 3
n_mol = length(x_init) ÷ 6
template_mol = reshape(template_mol,(3,n_atoms_per_mol))
radii = vcat([template_radii for i in 1:n_mol]...);

β = 1.0 / T
pf = MorphoMol.Energies.get_prefactors(rs, η)
Σ = vcat([[σ_r, σ_r, σ_r, σ_t, σ_t, σ_t] for _ in 1:n_mol]...)

energy(x) = solvation_free_energy_and_measures_in_bounds(x, template_mol, radii, rs, pf, 0.0, overlap_slope, persistence_weight, bnds, delaunay_eps)
perturbation(x) = perturb_single_randomly_chosen(x, σ_r, σ_t)
#perturbation(x) = perturb_all(x, Σ)

rwm = MorphoMol.Algorithms.RandomWalkMetropolis(energy, perturbation, β)

input = MorphoMol.Algorithms.MorphometricSimulationInput(
    template_mol,
    template_radii,
    n_mol,
    σ_r,
    σ_t,
    rs,
    η,
    pf,
    0.0,
    overlap_slope,
    T,
    0.0,
    0
)

energy_measures = Dict(
    "Es" => Vector{Float64}([]), 
    "Vs" => Vector{Float64}([]), 
    "As" => Vector{Float64}([]), 
    "Cs" => Vector{Float64}([]), 
    "Xs" => Vector{Float64}([]),
    "OLs" => Vector{Float64}([]),
    "CDGMs" => Vector{Any}([]),
    #"DGM3s" => Vector{Matrix{Float64}}([]),
    "CP2s" => Vector{Float64}([]),
    "CP3s" => Vector{Float64}([])
    )

algo_measures = Dict(
    "αs" => Vector{Float32}([])
)

output = MorphoMol.Algorithms.SimulationOutput(
    Vector{Vector{Float64}}([]),
    energy_measures,
    algo_measures
)

# MorphoMol.Algorithms.simulate!(rwm, output, deepcopy(x_init), simulation_time_minutes);

# output_directory = "assets/output/persistence_testerino/"
# name = "a"
# mkpath(output_directory)
# @save "$(output_directory)/$(name).jld2" input output

MorphoMol.Algorithms.SimulationOutput(Vector{Float64}[], Dict{String, Any}("Xs" => Float64[], "Cs" => Float64[], "OLs" => Float64[], "CP3s" => Float64[], "CP2s" => Float64[], "Vs" => Float64[], "Es" => Float64[], "As" => Float64[], "CDGMs" => Any[]), Dict{String, Any}("αs" => Float32[]))

In [8]:
@load "assets/output/persistence_testerino/exploding_persistence.jld2" input output

2-element Vector{Symbol}:
 :input
 :output

In [9]:
Es = output.energy_measures["Es"]
xs = [i for i in 1:length(Es)]
Vs = output.energy_measures["Vs"]
As = output.energy_measures["As"]
Cs = output.energy_measures["Cs"]
Xs = output.energy_measures["Xs"]
OLs = output.energy_measures["OLs"]
CDGMs = output.energy_measures["CDGMs"]
cp2s = output.energy_measures["CP2s"]
cp3s = output.energy_measures["CP3s"]
exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY["6r7m"]["template_mol"]
exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY["6r7m"]["state"]
thetas = [MorphoMol.Utilities.average_offset_distance(exp_template_centers, input.template_mol, exp_state, state) for state in output.states]
cdgms2 = [cdgm[2] for cdgm in CDGMs]
cdgms3 = [cdgm[3] for cdgm in CDGMs]
realizations = [MorphoMol.Utilities.get_matrix_realization(state, input.template_mol) for state in output.states];

In [11]:
f = Figure(fontsize = 7)
pd_ax_a = Axis(f[1, 1], title = "Persistence codim = 2")
pd_ax_b = Axis(f[1, 2], title = "Persistence codim = 3")
conf_ax = Axis3(f[1, 3])
tp_ax = Axis(f[2, 1], title = "Total Persistence")
theta_ax = Axis(f[2, 2], title = L"\Theta")
fsol_ax = Axis(f[2, 3], title = L"F_{sol}")

sl_i = Slider(f[3, 1:3], range = 1:length(Es), startvalue = 1)

x = sl_i.value
tps = persistence_weight * (cp2s + cp3s)
tp_mark = @lift(Point2f($x, $@lift(tps[$x])))
theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
fsol_mark = @lift(Point2f($x, $@lift(Es[$x])))

dgm_points_2 = @lift([Point2f(cdgms2[$x][i,1], cdgms2[$x][i,2]) for i in 1:size(cdgms2[$x])[1]])
dgm_points_3 = @lift([Point2f(cdgms3[$x][i,1], cdgms3[$x][i,2]) for i in 1:size(cdgms3[$x])[1]])

c1_points = @lift([Point3f(realizations[$x][1][1,i], realizations[$x][1][2,i], realizations[$x][1][3,i]) for i in 1:size(realizations[$x][1])[2]])
c2_points = @lift([Point3f(realizations[$x][2][1,i], realizations[$x][2][2,i], realizations[$x][2][3,i]) for i in 1:size(realizations[$x][2])[2]])

ms = 5
scatter!(pd_ax_a, dgm_points_2, color = :orange, markersize = ms)
scatter!(pd_ax_b, dgm_points_3, color = :green, markersize = ms)

scatter!(tp_ax, xs, tps, color=:red, markersize = ms)
scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

scatter!(conf_ax, c1_points, markersize = 15)
scatter!(conf_ax, c2_points, markersize = 15)

scatter!(theta_ax, xs, thetas, color=:magenta, markersize = ms)
scatter!(theta_ax, theta_mark, color = :black, markersize = 10)

scatter!(fsol_ax, xs, Es, color=:orange, markersize = ms)
scatter!(fsol_ax, fsol_mark, color = :black, markersize = 10)

display(f)


GLMakie.Screen(...)

In [21]:
rm("assets/output/persistence_testerino/polys/", recursive = true)
mkpath("assets/output/persistence_testerino/polys/")
radii = [input.template_radii; input.template_radii]
for (i, state) in enumerate(output.states)
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input.template_mol), radii, "assets/output/persistence_testerino/polys/$(i)", input.number_of_molecules, length(input.template_radii))
end